In [1]:
!uv pip install evaluate

Using Python 3.11.11 environment at: /teamspace/studios/this_studio/.conda


Resolved 33 packages in 365ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/189.08 KiB          
⠙ Preparing packages... (0/1)-------------- 16.00 KiB/189.08 KiB        
⠙ Preparing packages... (0/1)-------------- 32.00 KiB/189.08 KiB        
⠙ Preparing packages... (0/1)-------------- 48.00 KiB/189.08 KiB        
⠙ Preparing packages... (0/1)-------------- 61.06 KiB/189.08 KiB        
⠙ Preparing packages... (0/1)-------------- 77.06 KiB/189.08 KiB        
⠙ Preparing packages... (0/1)-------------- 93.06 KiB/189.08 KiB        
⠙ Preparing packages... (0/1)2m------------ 109.06 KiB/189.08 KiB       
⠙ Preparing packages... (0/1)---------- 125.06 KiB/189.08 KiB       
⠙ Preparing packages... (0/1)---------- 141.06 KiB/189.08 KiB       
⠙ Preparing packages... (0/1)---------- 157.06 KiB/189.08 KiB       
⠙ Preparing packages... (0/1)---------- 173.06 KiB/189.08 

### 1. Case Example with Yelp Dataset

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import pandas as pd
import evaluate

In [3]:
# Prepare and tokenize dataset
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:
# investigate the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [5]:
# preview the first 5 rows
df = pd.DataFrame(dataset["train"][:10])
pd.set_option('display.max_colwidth', None)
df.head(5)

,label,text
0,4,"dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
1,1,"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff. It seems that his staff simply never answers the phone. It usually takes 2 hours of repeated calling to get an answer. Who has time for that or wants to deal with it? I have run into this problem with many other doctors and I just don't get it. You have office workers, you have patients with medical needs, why isn't anyone answering the phone? It's incomprehensible and not work the aggravation. It's with regret that I feel that I have to give Dr. Goldberg 2 stars."
2,3,"Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."
3,3,Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June. He will be missed very much. \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
4,0,"I don't know what Dr. Goldberg was like before moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns your call. Both my adult children and husband have decided to leave this practice after experiencing such frustration. The entire office has an attitude like they are doing you a favor. Give me a break! Stay away from this doc and the practice. You deserve better and they will not be there when you really need them. I have never felt compelled to write a bad review about anyone until I met this pathetic excuse for a doctor who is all about the money."


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))

small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
# Setup evaluation 
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:

# Load pretrained model and evaluate model after each epoch
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

### 2. Use case 2

In [16]:
!uv pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.11.11 environment at: /teamspace/studios/this_studio/.conda
Resolved 5 packages in 151ms                                         
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)--------------     0 B/99.82 KiB           
⠙ Preparing packages... (0/2)-------------- 16.00 KiB/99.82 KiB         
⠙ Preparing packages... (0/2)-------------- 32.00 KiB/99.82 KiB         
⠙ Preparing packages... (0/2)-------------- 48.00 KiB/99.82 KiB         
⠙ Preparing packages... (0/2)---------- 64.00 KiB/99.82 KiB         
⠙ Preparing packages... (0/2)---------- 64.00 KiB/99.82 KiB         
click                ------------------------------ 64.00 KiB/99.82 KiB
⠙ Preparing packages... (0/2)--------------     0 B/1.44 MiB            
click                ------------------------------ 64.00 KiB/99.82 KiB
⠙ Preparing packages... (0/2)-------------- 14.91 KiB/1.44 MiB          
click                ------------------------------ 80

In [8]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [9]:
billsum = load_dataset("billsum", split="ca_test").shuffle(seed=42).select(range(200))

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [10]:
# print ths dataset
print(billsum)

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 200
})


In [11]:
billsum_df = pd.DataFrame(billsum[:10])
pd.set_option('display.max_colwidth', None)
billsum_df.head(2)

text  \
0  The people of the State of California do enact as follows:\n\n\nSECTION 1.\n(a) The Legislature finds and declares all of the following:\n(1) Short-lived climate pollutants, such as black carbon, fluorinated gases, and methane, are powerful climate forcers that have a dramatic and detrimental effect on air quality, public health, and climate change.\n(2) These pollutants create a warming influence on the climate that is many times more potent than that of carbon dioxide.\n(3) Short-lived climate pollutants that are toxic air contaminants also are a significant environmental risk factor for premature death.\n(4) Reducing emissions of these pollutants can have an immediate beneficial impact on climate change and on public health.\n(5) To the extent possible, efforts to reduce emissions of short-lived climate pollutants should focus on areas of the state that are disproportionately affected by poor air quality.\n(b) It is the intent of the Legislature to support the adoption of policies that improve organics recycling and innovative, cost effective, and environmentally beneficial uses of biomethane derived from solid waste facilities.\n(c) It is intent of the Legislature that the disposal reduction targets established pursuant to Section 39730.6 of the Health and Safety Code shall serve as a statewide average target and not as a minimum requirement for each jurisdiction.\nSEC. 2.\nSection 39730.5 is added to the Health and Safety Code, to read:\n39730.5.\n(a) No later than January 1, 2018, the state board shall approve and begin implementing the comprehensive short-lived climate pollutant strategy developed pursuant to Section 39730 to achieve a reduction in the statewide emissions of methane by 40 percent, hydrofluorocarbon gases by 40 percent, and anthropogenic black carbon by 50 percent below 2013 levels by 2030.\n(b) Prior to approving the short-lived climate pollutant strategy pursuant to subdivision (a), the state board shall do all of the following:\n(1) Coordinate with other state and local agencies and districts to develop measures identified as part of the strategy.\n(2) Provide a forum for public engagement by holding at least three public hearings in geographically diverse locations throughout the state.\n(3) Evaluate the best-available scientific, technological, and economic information to ensure that the strategy is cost effective and technologically feasible.\n(4) Incorporate and prioritize, as appropriate, measures and actions that provide the following cobenefits:\n(A) Job growth and local economic benefits in the state.\n(B) Public health benefits.\n(C) Potential for new innovation in technology, energy, and resource management practices.\n(c) The state board shall publicly notice the strategy described in subdivision (a) and post a copy of that strategy on the state board’s Internet Web site at least one month prior to the state board approving the strategy pursuant to subdivision (a).\nSEC. 3.\nSection 39730.6 is added to the Health and Safety Code, to read:\n39730.6.\n(a) Consistent with Section 39730.5, methane emissions reduction goals shall include the following targets to reduce the landfill disposal of organics:\n(1) A 50-percent reduction in the level of the statewide disposal of organic waste from the 2014 level by 2020.\n(2) A 75-percent reduction in the level of the statewide disposal of organic waste from the 2014 level by 2025.\n(b) Except as provided in this section and Section 42652.5 of the Public Resources Code, the state board shall not adopt, prior to January 1, 2025, requirements to control methane emissions associated with the disposal of organic waste in landfills other than through landfill methane emissions control regulations.\nSEC. 4.\nSection 39730.7 is added to the Health and Safety Code, to read:\n39730.7.\n(a) For purposes of this section, the following terms have the following meanings:\n(1) “Department” means the Department of Food and Agriculture.\n(2) “Commission” me

In [12]:
billsum = billsum.train_test_split(test_size=0.2)
print(billsum)

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 160
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 40
    })
})


In [13]:
# Prepare and tokenize dataset
tokenizer = AutoTokenizer.from_pretrained("t5-small")
prefix = "summarize: "

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [14]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [15]:
!uv pip install rouge_score absl-py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.11.11 environment at: /teamspace/studios/this_studio/.conda
Audited 2 packages in 805ms


In [16]:

# Setup evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [17]:
# Load pretrained model and evaluate model after each epoch
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [18]:
import transformers
print(transformers.__version__)

4.52.4


In [22]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=True,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_3747/2592845969.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.382466,0.126801,0.034005,0.103956,0.109618
2,No log,3.292472,0.122655,0.029674,0.101376,0.106561


TrainOutput(global_step=20, training_loss=3.578030014038086, metrics={'train_runtime': 23.68, 'train_samples_per_second': 13.514, 'train_steps_per_second': 0.845, 'total_flos': 86618752942080.0, 'train_loss': 3.578030014038086, 'epoch': 2.0})